In [0]:
import os

In [2]:
# 今回使用するレポジトリをGoogle Colabの環境にクローン
!git clone https://github.com/s0yamazaki/WallClassification.git

Cloning into 'WallClassification'...
remote: Counting objects: 31398, done.
remote: Total 31398 (delta 0), reused 0 (delta 0), pack-reused 31398
Receiving objects: 100% (31398/31398), 331.74 MiB | 40.30 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [3]:
# メインスクリプトである`model.py`の使用方法を表示
!python model.py -h

python3: can't open file 'model.py': [Errno 2] No such file or directory


In [4]:
# メインスクリプトの格納ディレクトリに移動
os.chdir('WallClassification')
!ls

classes_extractor.py  googleImageDownloader.json  resize.py
continue_fit.py       model.py			  semi-teching.py
crop.py		      resized_cleaned		  WallClassification.ipynb
erase_duplicates.py   resized_val


In [0]:
# メインスクリプトを実行
# !python model.py testmodel

In [5]:
# 生成されたファイルとディレクトリを表示
print('* カレントディレクトリ')
!ls .
print('\n* testmodel ディレクトリ')
!ls ./testmodel

* カレントディレクトリ
classes_extractor.py  googleImageDownloader.json  resize.py
continue_fit.py       model.py			  semi-teching.py
crop.py		      resized_cleaned		  WallClassification.ipynb
erase_duplicates.py   resized_val

* testmodel ディレクトリ
ls: cannot access './testmodel': No such file or directory


In [6]:
"""
サンプルのモデルの定義と訓練を行うモジュール
vgg16をベースにした転移学習モデルとなっている。
"""
from keras.models import Model,Sequential
import tensorflow as tf
from keras.layers.convolutional import Conv2D
from keras.layers import Activation, Dense, GlobalAveragePooling2D, Input, InputLayer, Lambda, Dropout, BatchNormalization,LeakyReLU
from keras.backend import sigmoid
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.callbacks import CSVLogger
from keras.callbacks import TensorBoard, EarlyStopping
import sys
import numpy as np
from collections import Counter
import continue_fit as cf
from keras.utils import multi_gpu_model
from keras import regularizers
from keras.layers.advanced_activations import LeakyReLU
from keras import backend as K

# Swishの定義
# http://yusuke-ujitoko.hatenablog.com/entry/2017/10/21/131232
# https://qiita.com/kiakiraki/items/2ec24f13ee870e13a0fb
def swish(x):
    return x * K.sigmoid(x)

def vgg_based_model(input_shape, n_categories, fulltraining = False):
    """
    VGG16をベースにした転移学習モデルを生成する。
    fulltraining: Trueにすると、ベースモデルも含めて訓練可能にする。訓練速度が非常に遅くなる。
    """
    base_model=VGG16(weights='imagenet',include_top=False,
                    input_tensor=Input(shape=input_shape))

    #add new layers instead of FC networks
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(1024)(x)
    x=BatchNormalization()(x)
    x=Activation(swish)(x)
    x=Dropout(0.6)(x)
    x=Dense(1024)(x)
    x=BatchNormalization()(x)
    x=Activation(swish)(x)
    x=Dropout(0.6)(x)
    prediction=Dense(n_categories,activation='softmax')(x)
    model=Model(inputs=base_model.input,outputs=prediction)

    if not fulltraining:
        # fix weights before VGG16 14layers
        for layer in base_model.layers[:15]:
            layer.trainable=False
    return model


Using TensorFlow backend.


In [8]:
# コマンドライン引数の定義/評価
batch_size=32
input_shape = (224,224,3)
#  parser = argparse.ArgumentParser()
# parser.add_argument("model_name", help="保存するモデルファイルの名前、兼tensorBoardのログディレクトリ名")
# parser.add_argument("-t", "--train_dir", default='resized_cleaned', help="トレーニングデータセットが入っているディレクトリ")
# parser.add_argument("-v","--validation_dir",default='resized_val', help ="バリデーションデータセットが入っているディレクトリ")
# args = parser.parse_args()


file_name = 'test09' #args.model_name
train_dir= 'resized_cleaned' # args.train_dir
validation_dir= 'resized_val' # args.validation_dir

#訓練データの読み込み及びデータ拡張を行うための画像ジェネレータを生成。
#VGG16用の前処理及び平行移動、回転、左右反転、シアー変換をランダムにかける。
train_datagen=ImageDataGenerator(
    preprocessing_function=preprocess_input, #VGG16の前処理
    height_shift_range=0.02,
    width_shift_range=0.02,
    shear_range=0.05,
    zoom_range=0.05,
    rotation_range=5,
    horizontal_flip=True,
    )
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[0:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
    )


n_categories=len(train_generator.class_indices)
#サンプルの多いクラスに予測が集中しないように、少ないサンプルのクラスほど重くなるように重みづけ
class_weight ={ clss: len(train_generator.classes) / len(train_generator.class_indices) / count
                 for (clss,count) in Counter(train_generator.classes).most_common() }
print("classes:",train_generator.class_indices)
print("class weight:",class_weight)

#バリデーションデータの画像読み込み処理を行うジェネレータを生成
validation_datagen=ImageDataGenerator(
    preprocessing_function=preprocess_input,
    )
validation_generator=validation_datagen.flow_from_directory(
    validation_dir,
    target_size=input_shape[0:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

model = vgg_based_model(input_shape, n_categories)
# parallel_model = multi_gpu_model(model, gpus=2)   #マルチGPUを使うときはこちら

model.compile(optimizer=Adam(lr=1e-3),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

#訓練(中断しても続きから継続できる)
hist=model.fit_generator(
    train_generator,
    epochs=100,
    #initial_epoch=cf.load_epoch_init(file_name), # 同じモデル名でも結果を引き継がないようにコメントアウト
    # use_multiprocessing=True,
    verbose=1,
    workers=8,
    validation_data=validation_generator,
    class_weight=class_weight,
    callbacks=[
        CSVLogger(file_name+'.csv'),
        TensorBoard(file_name),
        cf.early_stopping(model, file_name),
        ])

Found 5234 images belonging to 11 classes.
classes: {'ALC板': 0, 'ガラスカーテンウォール': 1, 'ガラスブロック': 2, 'コンクリートブロック': 3, 'コンクリート打ち放し': 4, 'スパンドレル': 5, 'スレート波板張り': 6, 'タイル張り': 7, '吹付タイル': 8, '押出成形セメント板': 9, '石張り': 10}
class weight: {7: 0.38527788001472213, 1: 0.48701963338606125, 3: 0.6000229278917804, 4: 0.8994672624162227, 5: 0.967110125646711, 0: 1.310793889306286, 2: 1.6295143212951433, 9: 2.0687747035573123, 8: 2.44009324009324, 10: 5.4070247933884295, 6: 11.895454545454545}
Found 1020 images belonging to 11 classes.
Epoch 1/100
161/164 [============================>.] - ETA: 1s - loss: 2.5159 - acc: 0.2284

164/164 [==============================] - 85s 515ms/step - loss: 2.5106 - acc: 0.2282 - val_loss: 3.4377 - val_acc: 0.1500
Epoch 2/100
164/164 [==============================] - 78s 478ms/step - loss: 1.9340 - acc: 0.3428 - val_loss: 2.5502 - val_acc: 0.3392
Epoch 3/100
164/164 [==============================] - 79s 479ms/step - loss: 1.5807 - acc: 0.4426 - val_loss: 2.2835 - val_acc: 0.3157
Epoch 4/100
 11/164 [=>............................] - ETA: 53s - loss: 1.3349 - acc: 0.4858

164/164 [==============================] - 78s 476ms/step - loss: 1.3583 - acc: 0.5205 - val_loss: 1.5018 - val_acc: 0.5343
Epoch 5/100
164/164 [==============================] - 78s 477ms/step - loss: 1.2780 - acc: 0.5647 - val_loss: 1.7902 - val_acc: 0.4510
Epoch 6/100
147/164 [=========================>....] - ETA: 7s - loss: 1.0991 - acc: 0.6134

164/164 [==============================] - 78s 475ms/step - loss: 1.1030 - acc: 0.6104 - val_loss: 1.5012 - val_acc: 0.5500
Epoch 7/100
164/164 [==============================] - 78s 478ms/step - loss: 0.9633 - acc: 0.6453 - val_loss: 1.2924 - val_acc: 0.5931
Epoch 8/100
164/164 [==============================] - 79s 483ms/step - loss: 0.9348 - acc: 0.6604 - val_loss: 1.3387 - val_acc: 0.5735
Epoch 9/100
  9/164 [>.............................] - ETA: 53s - loss: 0.8170 - acc: 0.7083

164/164 [==============================] - 79s 479ms/step - loss: 0.8140 - acc: 0.6922 - val_loss: 1.7545 - val_acc: 0.5206
Epoch 10/100
164/164 [==============================] - 78s 477ms/step - loss: 0.7935 - acc: 0.7105 - val_loss: 1.2687 - val_acc: 0.6118
Epoch 11/100
145/164 [=========================>....] - ETA: 7s - loss: 0.7086 - acc: 0.7369

...

164/164 [==============================] - 78s 477ms/step - loss: 0.0980 - acc: 0.9702 - val_loss: 1.8388 - val_acc: 0.7059
Epoch 90/100
164/164 [==============================] - 79s 481ms/step - loss: 0.0410 - acc: 0.9836 - val_loss: 1.9519 - val_acc: 0.6961
Epoch 91/100
144/164 [=========================>....] - ETA: 8s - loss: 0.0245 - acc: 0.9918

164/164 [==============================] - 79s 480ms/step - loss: 0.0297 - acc: 0.9909 - val_loss: 1.7356 - val_acc: 0.7196
Epoch 92/100
164/164 [==============================] - 79s 482ms/step - loss: 0.0289 - acc: 0.9897 - val_loss: 1.8404 - val_acc: 0.7078
Epoch 93/100
164/164 [==============================] - 79s 479ms/step - loss: 0.0312 - acc: 0.9905 - val_loss: 2.0057 - val_acc: 0.7000
Epoch 94/100
  7/164 [>.............................] - ETA: 53s - loss: 0.0887 - acc: 0.9777

164/164 [==============================] - 78s 479ms/step - loss: 0.0327 - acc: 0.9891 - val_loss: 2.0281 - val_acc: 0.6784
Epoch 95/100
164/164 [==============================] - 79s 480ms/step - loss: 0.0518 - acc: 0.9878 - val_loss: 2.0129 - val_acc: 0.6902
Epoch 96/100
144/164 [=========================>....] - ETA: 8s - loss: 0.0280 - acc: 0.9890

164/164 [==============================] - 79s 480ms/step - loss: 0.0290 - acc: 0.9890 - val_loss: 1.8273 - val_acc: 0.7147
Epoch 97/100
164/164 [==============================] - 79s 480ms/step - loss: 0.0383 - acc: 0.9889 - val_loss: 1.9259 - val_acc: 0.7098
Epoch 98/100
164/164 [==============================] - 79s 482ms/step - loss: 0.0452 - acc: 0.9842 - val_loss: 1.9060 - val_acc: 0.7157
Epoch 99/100
  7/164 [>.............................] - ETA: 53s - loss: 0.0107 - acc: 0.9911

164/164 [==============================] - 79s 480ms/step - loss: 0.0358 - acc: 0.9888 - val_loss: 1.8146 - val_acc: 0.7029
Epoch 100/100
164/164 [==============================] - 79s 482ms/step - loss: 0.0327 - acc: 0.9880 - val_loss: 1.9830 - val_acc: 0.6922


In [0]:
"""
サンプルのモデルの定義と訓練を行うモジュール
vgg16をベースにした転移学習モデルとなっている。
"""
from keras.models import Model,Sequential
import tensorflow as tf
from keras.layers.convolutional import Conv2D
from keras.layers import Activation, Dense, GlobalAveragePooling2D, Input, InputLayer, Lambda, Dropout, BatchNormalization,LeakyReLU
from keras.backend import sigmoid
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD, Adam
from keras.callbacks import CSVLogger
from keras.callbacks import TensorBoard, EarlyStopping
import sys
import numpy as np
from collections import Counter
import continue_fit as cf
from keras.utils import multi_gpu_model
from keras import regularizers
from keras.layers.advanced_activations import LeakyReLU

def vgg_based_model(input_shape, n_categories, fulltraining = False):
    """
    VGG16をベースにした転移学習モデルを生成する。
    fulltraining: Trueにすると、ベースモデルも含めて訓練可能にする。訓練速度が非常に遅くなる。
    """
    base_model=VGG16(weights='imagenet',include_top=False,
                    input_tensor=Input(shape=input_shape))

    #add new layers instead of FC networks
    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    x=Dense(1024)(x)
    x=BatchNormalization()(x)
    x=Activation(LeakyReLU())(x)
    x=Dropout(0.6)(x)
    x=Dense(1024)(x)
    x=BatchNormalization()(x)
    x=Activation(LeakyReLU())(x)
    x=Dropout(0.6)(x)
    prediction=Dense(n_categories,activation='softmax')(x)
    model=Model(inputs=base_model.input,outputs=prediction)

    if not fulltraining:
        # fix weights before VGG16 14layers
        for layer in base_model.layers[:15]:
            layer.trainable=False
    return model


In [13]:
# コマンドライン引数の定義/評価
batch_size=32
input_shape = (224,224,3)
#  parser = argparse.ArgumentParser()
# parser.add_argument("model_name", help="保存するモデルファイルの名前、兼tensorBoardのログディレクトリ名")
# parser.add_argument("-t", "--train_dir", default='resized_cleaned', help="トレーニングデータセットが入っているディレクトリ")
# parser.add_argument("-v","--validation_dir",default='resized_val', help ="バリデーションデータセットが入っているディレクトリ")
# args = parser.parse_args()


file_name = 'test10' #args.model_name
train_dir= 'resized_cleaned' # args.train_dir
validation_dir= 'resized_val' # args.validation_dir

#訓練データの読み込み及びデータ拡張を行うための画像ジェネレータを生成。
#VGG16用の前処理及び平行移動、回転、左右反転、シアー変換をランダムにかける。
train_datagen=ImageDataGenerator(
    preprocessing_function=preprocess_input, #VGG16の前処理
    height_shift_range=5,
    width_shift_range=5,
    shear_range=0.05,
    zoom_range=0.95,
    rotation_range=7,
    horizontal_flip=True,
    )
train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=input_shape[0:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
    )


n_categories=len(train_generator.class_indices)
#サンプルの多いクラスに予測が集中しないように、少ないサンプルのクラスほど重くなるように重みづけ
class_weight ={ clss: len(train_generator.classes) / len(train_generator.class_indices) / count
                 for (clss,count) in Counter(train_generator.classes).most_common() }
print("classes:",train_generator.class_indices)
print("class weight:",class_weight)

#バリデーションデータの画像読み込み処理を行うジェネレータを生成
validation_datagen=ImageDataGenerator(
    preprocessing_function=preprocess_input,
    )
validation_generator=validation_datagen.flow_from_directory(
    validation_dir,
    target_size=input_shape[0:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

model = vgg_based_model(input_shape, n_categories)
# parallel_model = multi_gpu_model(model, gpus=2)   #マルチGPUを使うときはこちら

model.compile(optimizer=Adam(lr=1e-3),
            loss='categorical_crossentropy',
            metrics=['accuracy'])

#訓練(中断しても続きから継続できる)
hist=model.fit_generator(
    train_generator,
    epochs=100,
    #initial_epoch=cf.load_epoch_init(file_name), # 同じモデル名でも結果を引き継がないようにコメントアウト
    # use_multiprocessing=True,
    verbose=1,
    workers=8,
    validation_data=validation_generator,
    class_weight=class_weight,
    callbacks=[
        CSVLogger(file_name+'.csv'),
        TensorBoard(file_name),
        # cf.early_stopping(model, file_name),
        ])

Found 5234 images belonging to 11 classes.
classes: {'ALC板': 0, 'ガラスカーテンウォール': 1, 'ガラスブロック': 2, 'コンクリートブロック': 3, 'コンクリート打ち放し': 4, 'スパンドレル': 5, 'スレート波板張り': 6, 'タイル張り': 7, '吹付タイル': 8, '押出成形セメント板': 9, '石張り': 10}
class weight: {7: 0.38527788001472213, 1: 0.48701963338606125, 3: 0.6000229278917804, 4: 0.8994672624162227, 5: 0.967110125646711, 0: 1.310793889306286, 2: 1.6295143212951433, 9: 2.0687747035573123, 8: 2.44009324009324, 10: 5.4070247933884295, 6: 11.895454545454545}
Found 1020 images belonging to 11 classes.


/usr/local/lib/python3.6/dist-packages/keras/activations.py:115: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))


Epoch 1/100
164/164 [==============================] - 87s 527ms/step - loss: 3.1346 - acc: 0.1253 - val_loss: 2.3139 - val_acc: 0.2422
Epoch 2/100
164/164 [==============================] - 81s 494ms/step - loss: 2.5021 - acc: 0.2020 - val_loss: 3.6806 - val_acc: 0.1637
Epoch 3/100
129/164 [======================>.......] - ETA: 15s - loss: 2.5000 - acc: 0.1853

164/164 [==============================] - 83s 504ms/step - loss: 2.4294 - acc: 0.2031 - val_loss: 2.2228 - val_acc: 0.3529
Epoch 4/100
164/164 [==============================] - 84s 511ms/step - loss: 2.0443 - acc: 0.2688 - val_loss: 2.5159 - val_acc: 0.2598
Epoch 5/100
164/164 [==============================] - 81s 495ms/step - loss: 2.1067 - acc: 0.2688 - val_loss: 2.0744 - val_acc: 0.3598
Epoch 6/100
  6/164 [>.............................] - ETA: 55s - loss: 1.7584 - acc: 0.3542

164/164 [==============================] - 82s 503ms/step - loss: 1.9633 - acc: 0.3264 - val_loss: 1.8659 - val_acc: 0.3451
Epoch 7/100
164/164 [==============================] - 82s 500ms/step - loss: 1.8616 - acc: 0.3562 - val_loss: 2.0443 - val_acc: 0.4069
Epoch 8/100
146/164 [=========================>....] - ETA: 7s - loss: 1.7183 - acc: 0.4018

...

164/164 [==============================] - 77s 469ms/step - loss: 0.5355 - acc: 0.7944 - val_loss: 1.3752 - val_acc: 0.7039
Epoch 89/100
164/164 [==============================] - 75s 459ms/step - loss: 0.5317 - acc: 0.7959 - val_loss: 1.3218 - val_acc: 0.7314
Epoch 90/100
164/164 [==============================] - 77s 470ms/step - loss: 0.5424 - acc: 0.7868 - val_loss: 1.3685 - val_acc: 0.7245
Epoch 91/100
  7/164 [>.............................] - ETA: 54s - loss: 0.5478 - acc: 0.7946

164/164 [==============================] - 76s 462ms/step - loss: 0.5493 - acc: 0.7948 - val_loss: 1.3845 - val_acc: 0.7294
Epoch 92/100
164/164 [==============================] - 77s 472ms/step - loss: 0.4939 - acc: 0.8012 - val_loss: 1.4533 - val_acc: 0.7206
Epoch 93/100
144/164 [=========================>....] - ETA: 8s - loss: 0.5247 - acc: 0.8020

164/164 [==============================] - 76s 464ms/step - loss: 0.5293 - acc: 0.7981 - val_loss: 1.5584 - val_acc: 0.6951
Epoch 94/100
164/164 [==============================] - 76s 462ms/step - loss: 0.5256 - acc: 0.7963 - val_loss: 1.5867 - val_acc: 0.6853
Epoch 95/100
164/164 [==============================] - 76s 465ms/step - loss: 0.5123 - acc: 0.7917 - val_loss: 1.5010 - val_acc: 0.7225
Epoch 96/100
  7/164 [>.............................] - ETA: 54s - loss: 0.4565 - acc: 0.8036

164/164 [==============================] - 78s 473ms/step - loss: 0.5059 - acc: 0.8097 - val_loss: 1.3112 - val_acc: 0.7294
Epoch 97/100
164/164 [==============================] - 76s 465ms/step - loss: 0.4986 - acc: 0.8058 - val_loss: 1.3231 - val_acc: 0.7363
Epoch 98/100
144/164 [=========================>....] - ETA: 8s - loss: 0.4918 - acc: 0.8066

164/164 [==============================] - 77s 471ms/step - loss: 0.4924 - acc: 0.8077 - val_loss: 1.2483 - val_acc: 0.7500
Epoch 99/100
164/164 [==============================] - 79s 479ms/step - loss: 0.5265 - acc: 0.8048 - val_loss: 1.3872 - val_acc: 0.7245
Epoch 100/100
164/164 [==============================] - 77s 469ms/step - loss: 0.4573 - acc: 0.8158 - val_loss: 1.4645 - val_acc: 0.7245
